In [14]:
from sqlalchemy import create_engine, inspect

def list_tables(db_url):
    engine = create_engine(db_url)
    inspector = inspect(engine)
    tables = inspector.get_table_names()
    return tables

# Example usage
db_url = 'sqlite:///fraud.db'  # URL for your SQLite database
tables = list_tables(db_url)
print("Tables:", tables)

Tables: []


In [13]:
import pandas as pd
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, Float, String, Boolean

# Step 1: Load CSV into DataFrame
df = pd.read_csv('fraud_data.csv')

# Step 2: Define a function to infer SQLAlchemy column types
def infer_column_type(series):
    if pd.api.types.is_integer_dtype(series):
        return Integer
    elif pd.api.types.is_float_dtype(series):
        return Float
    elif pd.api.types.is_bool_dtype(series):
        return Boolean
    elif pd.api.types.is_datetime64_any_dtype(series):
        return String  # Use String for datetime for simplicity; handle separately if needed
    else:
        return String

# Step 3: Create SQLAlchemy engine and initialize metadata
engine = create_engine('sqlite:///fraud.db',echo=True)

metadata = MetaData()

# Step 4: Define columns dynamically based on CSV data types
columns = [Column('id', Integer, primary_key=True)]  # Optional: add an ID column for primary key

for column_name in df.columns:
    column_type = infer_column_type(df[column_name])
    print(column_type)
    columns.append(Column(column_name, column_type))

# Step 5: Define the table
fraud_table = Table('fraud', metadata, *columns)

# Step 6: Create the table in the database
metadata.create_all(engine)

# Step 7: Write DataFrame to SQLite database
with engine.connect() as conn:
    df.to_sql('fraud', conn.connection, if_exists='replace', index=False)

with engine.connect() as conn:
    query = "SELECT * FROM fraud LIMIT 3"
    result = conn.execute(query)
    for row in result:
        print(row)

<class 'sqlalchemy.sql.sqltypes.String'>
<class 'sqlalchemy.sql.sqltypes.String'>
<class 'sqlalchemy.sql.sqltypes.String'>
<class 'sqlalchemy.sql.sqltypes.Float'>
<class 'sqlalchemy.sql.sqltypes.String'>
<class 'sqlalchemy.sql.sqltypes.String'>
<class 'sqlalchemy.sql.sqltypes.Float'>
<class 'sqlalchemy.sql.sqltypes.Float'>
<class 'sqlalchemy.sql.sqltypes.Integer'>
<class 'sqlalchemy.sql.sqltypes.String'>
<class 'sqlalchemy.sql.sqltypes.String'>
<class 'sqlalchemy.sql.sqltypes.String'>
<class 'sqlalchemy.sql.sqltypes.Float'>
<class 'sqlalchemy.sql.sqltypes.Float'>
<class 'sqlalchemy.sql.sqltypes.String'>
2024-08-09 22:20:29,523 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-09 22:20:29,526 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("fraud")
2024-08-09 22:20:29,528 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-09 22:20:29,537 INFO sqlalchemy.engine.Engine COMMIT


/var/folders/wb/wd79lvhj47g_7s7zw5ytxfzm0000gn/T/ipykernel_85363/1322103263.py:41: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('fraud', conn.connection, if_exists='replace', index=False)


2024-08-09 22:20:32,744 INFO sqlalchemy.engine.Engine SELECT * FROM fraud LIMIT 3
2024-08-09 22:20:32,746 INFO sqlalchemy.engine.Engine [raw sql] ()
('04-01-2019 00:58', '"Stokes, Christiansen and Sipes"', 'grocery_net', 14.37, 'Wales', 'AK', 64.7556, -165.6723, 145, '"Administrator, education"', '09-11-1939', 'a3806e984cec6ac0096d8184c64ad3a1', 65.654142, -164.722603, '1')
('04-01-2019 15:06', 'Predovic Inc', 'shopping_net', 966.11, 'Wales', 'AK', 64.7556, -165.6723, 145, '"Administrator, education"', '09-11-1939', 'a59185fe1b9ccf21323f581d7477573f', 65.468863, -165.473127, '1')
('04-01-2019 22:37', 'Wisozk and Sons', 'misc_pos', 49.61, 'Wales', 'AK', 64.7556, -165.6723, 145, '"Administrator, education"', '09-11-1939', '86ba3a888b42cd3925881fa34177b4e0', 65.347667, -165.914542, '1')
